# Etapes de la procédure

Étapes de la procédure:

1. Installation des bibliothèques nécessaires:
    - Nous commençons par installer toutes les bibliothèques nécessaires pour que notre script fonctionne. Cela inclut la gestion des PDFs, l'interaction avec OpenAI, l'indexation des vecteurs avec FAISS, et d'autres outils.

2. Configuration des clés API:
    - Ensuite, nous configurons l'accès à l'API OpenAI. Cela nous permettra d'utiliser les embeddings d'OpenAI pour nos besoins.

3. Chargement du fichier PDF:
    - Nous chargeons notre fichier PDF spécifique pour extraire le texte qu'il contient. Ceci sera la base de notre recherche et de notre interaction avec ChatGPT.

4. Prétraitement du texte:
    - Le texte extrait est ensuite divisé en morceaux plus petits pour s'assurer qu'il est gérable et pour éviter de dépasser les limites de taille des tokens.

5. Création des embeddings:
    - Nous convertissons ensuite ces morceaux de texte en embeddings (vecteurs) en utilisant OpenAI. Ces embeddings nous permettent de comparer la similitude entre différentes portions de texte et les questions posées.

6. Indexation avec FAISS:
    - Avec FAISS, nous indexons ces embeddings pour faciliter une recherche rapide. Lorsque nous posons une question, nous rechercherons les parties les plus pertinentes du texte en utilisant cet index.

7. Mise en place de la chaîne de traitement de la réponse aux questions:
    - Nous configurons la chaîne de traitement qui prendra nos questions, cherchera les textes pertinents, et générera des réponses à l'aide de ChatGPT.

8. Interaction:
    - Enfin, nous interagissons avec notre système en posant des questions, et le système nous donne des réponses basées sur le contenu du PDF.


# Importation des packages nécesseraire

In [12]:
# Installer langchain, une bibliothèque pour les chaînes de traitement du langage naturel.
!pip install langchain

# Installer la bibliothèque openai pour interagir avec les modèles d'OpenAI, comme GPT-3.
!pip install openai

# Installer PyPDF2, une bibliothèque pour lire, écrire et manipuler les fichiers PDF en Python.
!pip install PyPDF2

# Installer faiss-cpu, une bibliothèque pour l'indexation efficace et la recherche de similitude des vecteurs. 
!pip install faiss-cpu

# Installer tiktoken, une bibliothèque d'OpenAI pour compter les tokens dans un texte sans effectuer d'appel à l'API.
!pip install tiktoken




In [13]:
# Importation des bibliothèques nécessaires

# PyPDF2 est une bibliothèque pour lire, écrire et manipuler les fichiers PDF en Python.
from PyPDF2 import PdfReader

# OpenAIEmbeddings est une classe de la bibliothèque langchain pour télécharger et gérer les embeddings d'OpenAI.
from langchain.embeddings.openai import OpenAIEmbeddings

# CharacterTextSplitter est une classe pour diviser de grands textes en morceaux plus petits, utile pour gérer des limites de tokens ou des contraintes de taille.
from langchain.text_splitter import CharacterTextSplitter

# FAISS est une bibliothèque pour l'indexation efficace et la recherche de similitude des vecteurs. Ici, nous importons l'interface de langchain pour FAISS.
from langchain.vectorstores import FAISS

# load_qa_chain est une fonction pour charger une chaîne de traitement préconfigurée pour répondre aux questions à partir de textes.
from langchain.chains.question_answering import load_qa_chain

# OpenAI est une classe pour interagir avec les modèles d'OpenAI, tels que GPT-3, via la bibliothèque langchain.
from langchain.llms import OpenAI

# os est une bibliothèque standard de Python pour interagir avec le système d'exploitation, comme la gestion des variables d'environnement.
import os


# re est une bibliothèque standard de Python pour travailler avec les expressions régulières, utile pour la manipulation de texte et la recherche de motifs.
import re


# Configurer l'API OpenAI



Voici les étapes pour configurer l'API d'OpenAI :

1. Inscription sur OpenAI
Si vous n'avez pas encore de compte, inscrivez-vous sur [OpenAI](https://www.openai.com/).

2. Obtention de la clé API
    - Une fois connecté, rendez-vous dans la section **API** de votre tableau de bord.
    - Générez une nouvelle clé API ou utilisez une clé existante.

Pour générer sa clé API, il faut:
    - Cliquer sur le lien de son profil
    -Cliquer sur view API keys
    - Create new secret key ensuite donner un nom et copier le code

3. Ajouter une méthode de paiement, l'api ne fonctionne pas si vous avez 0 dollar (Billing)

4. Installation de la bibliothèque OpenAI Python
Installez le paquet officiel d'OpenAI pour Python. Vous pouvez le faire via pip :
import openai

openai.api_key = 'VOTRE_CLÉ_API'


In [14]:
# Configuration de l'API OpenAI
# Assurez-vous d'avoir votre clé API avant de poursuivre.
# Remplacez 'VOTRE_CLÉ_ICI' par votre clé.
import os
os.environ["OPENAI_API_KEY"] = "VOTRE_CLÉ_ICI"

# Importation et prétraitement des données

In [15]:
# Lecture et extraction du texte à partir du fichier PDF
# Assurez-vous que le fichier est présent dans le répertoire de travail ou fournissez le chemin complet
chemin_pdf = 'Societe-Generale-Pilier-3_T2-2022_FR.pdf'
lecteur = PdfReader(chemin_pdf)

lecteur


In [16]:
# Initialisation d'une chaîne vide pour stocker l'ensemble du texte extrait du PDF.
texte_brut = ''

# Boucle à travers chaque page du PDF. La fonction enumerate renvoie également un indice (i) pour chaque page.
for i, page in enumerate(lecteur.pages):
    
    # Extraction du texte de la page actuelle
    text = page.extract_text()
    
    # Vérification si du texte a été effectivement extrait de la page
    if text:
        
        # Ajout du texte extrait à la variable texte_brut
        texte_brut += text


In [17]:
texte_brut

'RAPP ORT  \nSUR LES RIS QUE S\nĀ\x8e¸\x8eKĄ\x03ј\x03ӳ\x03ј ѕӝѕћӝїѕїї2022ABRÉVIATIONS COURANTES\nMillions d’euros :M EUR/Millia rds d’euros : Md EUR/ETP :  Effectifs en équivalent temps pleinff\nClassements :  les sou rces des classements sont mentionn ées ex plicitement, à défaut, l’information est de sou rce interne.\x1c\x18\x16\x16\x0b\x13\x1b\x0f\n\x06 9\x0f\x14\x15\x12\x12\x1c\x11\x1d\x0f\x16!\x1d\n\n :\x12\r\x0f\x1e\x11\x1f\x1c\x1d\x10\x11\x1c\x15\x1d\x1b\x1f\x11\nTypologiedesrisques \x05\x02\x04 7\nFacteursderisque \x05\x02\x05 8\n\x05\x04 ;\x13\x11\x1d\x1e\x15\x19\x18\x10\x1f\x0f\r\x1a\x15\x1e\r\x16\x11\x1e\r\x10!\x1b\x1f\r\x1e\x15\x19\x18\n\x10\x11\x1d\x12\x19\x18\x10\x1d\x1a\x1c\x19\x1a\x1c\x11\x1d\nChampd’application–Périmètre \x06\x02\x04\nprudentiel 21\nExpositionspondéréesetexigences \x06\x02\x05\nde\xa0fonds\xa0propres 26\nRatioTLAC \x06\x02\x06 27\nRatiodelevier\x06\x02\x07 27\nCoussincontracyclique \x06\x02\x08 28\nInformationsquantitatives \x06\x02\t\ncomplémentairess

Si vous utilisez des modèles de traitement de texte comme ceux basés sur l'architecture Transformer (e.g., BERT, GPT, etc.), ils ont une limite maximale quant au nombre de tokens qu'ils peuvent gérer en une seule fois. Par exemple, le modèle BERT de base peut gérer jusqu'à 512 tokens par entrée. Si votre texte dépasse cette limite, il doit être découpé pour être traité.

Découper permet également de mieux générer la mémoire

In [18]:
# Découpage du texte en morceaux plus petits pour éviter les limites de taille de token
separateur = "\n"
decoupeur_texte = CharacterTextSplitter(
    separator=separateur,
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len
)
textes = decoupeur_texte.split_text(texte_brut)

In [19]:
textes

['RAPP ORT  \nSUR LES RIS QUE S\nĀ\x8e¸\x8eKĄ\x03ј\x03ӳ\x03ј ѕӝѕћӝїѕїї2022ABRÉVIATIONS COURANTES\nMillions d’euros :M EUR/Millia rds d’euros : Md EUR/ETP :  Effectifs en équivalent temps pleinff\nClassements :  les sou rces des classements sont mentionn ées ex plicitement, à défaut, l’information est de sou rce interne.\x1c\x18\x16\x16\x0b\x13\x1b\x0f\n\x06 9\x0f\x14\x15\x12\x12\x1c\x11\x1d\x0f\x16!\x1d\n :\x12\r\x0f\x1e\x11\x1f\x1c\x1d\x10\x11\x1c\x15\x1d\x1b\x1f\x11\nTypologiedesrisques \x05\x02\x04 7\nFacteursderisque \x05\x02\x05 8\n\x05\x04 ;\x13\x11\x1d\x1e\x15\x19\x18\x10\x1f\x0f\r\x1a\x15\x1e\r\x16\x11\x1e\r\x10!\x1b\x1f\r\x1e\x15\x19\x18\n\x10\x11\x1d\x12\x19\x18\x10\x1d\x1a\x1c\x19\x1a\x1c\x11\x1d\nChampd’application–Périmètre \x06\x02\x04\nprudentiel 21\nExpositionspondéréesetexigences \x06\x02\x05\nde\xa0fonds\xa0propres 26\nRatioTLAC \x06\x02\x06 27\nRatiodelevier\x06\x02\x07 27\nCoussincontracyclique \x06\x02\x08 28\nInformationsquantitatives \x06\x02\t\ncomplémentairessu

In [20]:
# Téléchargement des embeddings depuis OpenAI
modeles_embedding = OpenAIEmbeddings()

In [21]:
# Initialisation de la structure de recherche FAISS (Facebook AI Similarity Search") :
# 1. Chaque texte de la liste 'textes' est transformé en un vecteur (embedding) en utilisant le modèle 'modeles_embedding'.
# 2. Ces vecteurs sont ensuite stockés dans une structure de données FAISS optimisée pour la recherche rapide de similarité.
# 'docsearch' sera utilisé pour retrouver rapidement les documents les plus similaires à une entrée donnée basée sur la similarité sémantique.
docsearch = FAISS.from_texts(textes, modeles_embedding)


In [22]:
# Chargement de la chaîne de traitement spécifique à la réponse aux questions (QA) :
# 1. La fonction 'load_qa_chain' est utilisée pour initialiser une chaîne de traitement qui permet de répondre aux questions basées sur le contexte fourni.
# 2. Elle utilise le modèle 'OpenAI()' pour générer les réponses.
# 3. Le paramètre 'chain_type' spécifie le type ou la variante de la chaîne de traitement à charger, ici, c'est le type "stuff".
# Une fois chargée, 'chain' peut être utilisée pour traiter des questions et obtenir des réponses pertinentes en fonction du contexte donné.
chain = load_qa_chain(OpenAI(), chain_type="stuff")


In [23]:
# Exemple de requête pour chercher des informations dans le document
requete = "quel était le montant des RWA à la société générale?"
documents_similaires = docsearch.similarity_search(requete)
reponse = chain.run(input_documents=documents_similaires, question=requete)
print(reponse)

AuthenticationError: Incorrect API key provided: sk-5W201***************************************mkgu. You can find your API key at https://platform.openai.com/account/api-keys.

In [13]:
# Exemple de requête pour chercher des informations dans le document
requete = "Quels sont les principaux indicateurs de performance financière mentionnés dans le document ?"
documents_similaires = docsearch.similarity_search(requete)
reponse = chain.run(input_documents=documents_similaires, question=requete)
print(reponse)

 Les principaux indicateurs de performance financière mentionnés dans le document sont le ratio de financement stable net (%), les prêts et titres performants, les opérations de financement sur titres performantes avec des clients financiers garantis par des actifs liquides de qualité élevée de niveau 1 soumis à une décote de 0 %, les prêts performants à des entreprises non financières, les prêts performants à la clientèle de détail et aux petites entreprises et les prêts performants aux emprunteurs souverains et aux entités du secteur public.


In [14]:
# Exemple de requête pour chercher des informations dans le document
requete = "Qui est l'auteur du document? "
documents_similaires = docsearch.similarity_search(requete)
reponse = chain.run(input_documents=documents_similaires, question=requete)
print(reponse)

 Mme Claire DUMAS


In [15]:
# Exemple de requête pour chercher des informations dans le document
requete = "Quels sont les risques mentionnés dans le document? "
documents_similaires = docsearch.similarity_search(requete)
reponse = chain.run(input_documents=documents_similaires, question=requete)
print(reponse)

 Les risques mentionnés dans le document comprennent les risques liés aux contextes macroéconomiques, de marché et réglementaires, les risques de crédit et de contrepartie, les risques de marché et structurels, les dérivés et tout autre actif, les risques opérationnels, les risques de non-conformité, les risques de réputation, les risques de conduite inappropriée, les risques liés à la gouvernance des contreparties et des parties prenantes et les risques liés aux activités propres du Groupe, tels que les risques climatiques physiques et les risques sociaux et de gouvernance.


In [16]:
# Exemple de requête pour chercher des informations dans le document
requete = "Quel est le montant des fonds propres à la fin de la période de déclaration? "
documents_similaires = docsearch.similarity_search(requete)
reponse = chain.run(input_documents=documents_similaires, question=requete)
print(reponse)

 Le montant des fonds propres à la fin de la période de déclaration est de 56.024 MEUR en 30 juin 2022 et de 57.907 MEUR en 31 décembre 2021.
